# Analysis of the stock of vacancies
This script analyses the re-weighted stock of vacancies broken down by occupation, industry, location and skill category. It also does an analysis of location quotients by Travel To Work Areas (with associated Gini Indices) when analysing the breakdown by 'industry and location' and 'skill category and location'. 


A copy of the report can be found in this github repository.


(Please forgive the less than ideal coding practices)



# Imports and definitions

In [ ]:
# ------------------------ DEPENDENCIES AND FUNCTIONS ------------------------
from collections import Counter
from copy import deepcopy
import datetime
import folium
import gzip
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import pickle
import seaborn as sns
import sys
from time import time as tt
from shapely import geometry
from scipy.special import gammaln
import statsmodels.api as sm
import json

from utils_general import TaskTimer, print_elapsed, nesta_colours, sic_letter_to_text, flatten_lol, printdf, socnames_dict
from textkernel_load_utils import tk_params, DATA_PATH, create_tk_import_dict, read_and_append_chunks
from flow_to_stock_funcs import get_stock_breakdown, load_ons_vacancies
from maputils_pin import draw_map

timer = TaskTimer()
print('Done')



In [ ]:
# folder with all the figures for the report
output_folder = '/path/to/outputs'
local_data_path = '/path/to/local/data'

DATE_ID = 'July2020'


In [ ]:
SAVEFIG = False


In [ ]:
# Use more colours
aug_nesta_colours = nesta_colours + [[1., 0., 0.], [0., 0., 1.], [0.,1.,0.], [.7, .7, .7]]



In [ ]:
# SIC mapping between text and letters

sic_letter_to_text['Z'] = 'Others'
sic_letter_to_text['L_O_S'] = 'Personal and public services'#including non-profit and estate agents
sic_letter_to_text['D_E'] = 'Utilities (energy, water and waste)'
sic_letter_to_text['M_P'] = 'Educational and professional activities'
sic_letter_to_text['uncertain'] = 'Uncertain'

sic_text_to_letter= {}
for letter in sic_letter_to_text.keys():
    sic_text= sic_letter_to_text[letter]
    sic_text_to_letter[sic_text] = letter

sic_letter_to_text_abbr = deepcopy(sic_letter_to_text)
sic_letter_to_text_abbr['T'] = 'Activities of households as employers'

print('Done')


In [ ]:
def resample_soc_to_n_digits(soc_code,n=3):
    """ Resample SOC codes from 4 to n-digits"""
    if np.isnan(soc_code):
        return np.nan
    else:
        m = {1: 1000, 2: 100, 3: 10}[n]
        return (soc_code - soc_code%m)/m

def resample_soc_to_n_digits_df(soc_code_df,n=3):
    """ Resample SOC codes from 4 to n-digits for a dataframe"""
    m = {1: 1000, 2: 100, 3: 10}[n]
    return (soc_code_df - soc_code_df%m)/m
    
    

In [ ]:
#%% -------------------------------------------------------------------------
#              Main functions to convert from flow to stock
#% --------------------------------------------------------------------------

''' 
def set_month_at_beginning(x):
    """Set a datetime to the beginning of the month"""
    return pd.offsets.MonthBegin(0).rollback(x)

def set_month_at_end(x):
    """ Set a datetime to the end of the month"""
    return pd.offsets.MonthEnd(0).rollforward(x)
    
def get_stock_breakdown(data, agg_func = 'sum', agg_col = 'vacancy_weight', 
                              breakdown_col = 'organization_industry_value', BOUNDARY = None):
    """Compute the daily stock of vacancies via cumulative sum of net Flow.
    
    Keyword arguments:
    data -- dataframe with online job vacancies. Need to have "date", 
            "end_date" and agg_col columns
    agg_func: whether to count the vacancies or to sum the weights
    agg_col -- reference column to aggregate (usually column with per-vacancy weights)
    BOUNDARY -- what to do wrt boundary conditions (start and end month)
    """
    
    if not isinstance(breakdown_col,list):
        breakdown_col = [breakdown_col]
        
    start_day = data.date.min()
    end_day = data.date.max()
    
    print(agg_func)
    if agg_func == 'sum':
        vacancy_flow_per_day = data.groupby(['date'] + breakdown_col)[agg_col].sum()
        vacancy_remove_per_day = data.groupby(['end_date']+ breakdown_col)[agg_col].sum()
    else:
        vacancy_flow_per_day = data.groupby(['date'] + breakdown_col)[agg_col].count()
        vacancy_remove_per_day = data.groupby(['end_date'] + breakdown_col)[agg_col].count()
    
    for _ in range(len(breakdown_col)):
        vacancy_flow_per_day = vacancy_flow_per_day.unstack()
        vacancy_remove_per_day = vacancy_remove_per_day.unstack()
    
    # shift vacancy_remove_per_day by one day since vacancies disappear the day
    # after their expiration date
    vacancy_remove_per_day = vacancy_remove_per_day.shift(1)
    
    # adjust so that they start and end on the same dates
    vacancy_flow_per_day = vacancy_flow_per_day.reindex(pd.date_range(start=start_day,
                                        end=end_day,freq='D'), fill_value=0)#, level=0)
    vacancy_remove_per_day = vacancy_remove_per_day.reindex(pd.date_range(start=start_day,
                                        end=end_day,freq='D'), fill_value=0)#, level=0)
    
    # compute the net Flow
    net_flow = vacancy_flow_per_day.fillna(0) - vacancy_remove_per_day.fillna(0)
    
    # Get the daily stock
    daily_stock = net_flow.cumsum()
    
    # Resample to monthly stock
    monthly_stock = net_flow.resample('M').sum().cumsum()/2
    monthly_stock.index = monthly_stock.index.map(set_month_at_beginning)
    
    # enforce boundary conditions
    if BOUNDARY == 'valid':
        monthly_stock = monthly_stock[monthly_stock.index>=set_month_at_beginning(
            pd.to_datetime(FIRST_VALID_MONTH))]
    return monthly_stock, daily_stock, vacancy_flow_per_day, vacancy_remove_per_day

def load_ons_vacancies(start_date = '2015-03-01', end_date = '2019-10-31'):
    ons_df = pd.read_excel(f"{DATA_PATH}/data/aux/x06apr20.xls", sheet_name='Vacancies by industry',
                          skiprows = 3)
    # keep columns with data and clean the column names
    ons_df = ons_df[[col for col in ons_df.columns if not 'Unnamed:' in col]]
    cleaned_col_names = {}
    for col in ons_df.columns[2:]:
        cleaned_col = col.replace('&', 'and').replace('-','').replace(',','').lower()
        cleaned_col = ''.join([t for t in cleaned_col if not t.isdigit()])
        cleaned_col_names[col] = '_'.join(cleaned_col.split())
    # manual adjustment for one column
    cleaned_col_names['Manu-    facturing'] = 'manufacturing'
    cleaned_col_names['SIC 2007 sections'] = 'month'
    cleaned_col_names['All vacancies1 '] = 'vacancies'
    ons_df = ons_df.rename(columns = cleaned_col_names)
    # extract the row with the letters
    sic_letters = ons_df.iloc[0]
    # remove empty rows
    ons_df = ons_df.loc[(ons_df.month.notna()) & (ons_df.vacancies.notna())]
    # join up some industries
    ons_df = ons_df.assign(wholesale_retail_motor_trade_and_repair = 
                           ons_df.motor_trades + ons_df.wholesale + ons_df.retail)
    ons_df = ons_df.assign(wholesale_and_retail = ons_df.wholesale + ons_df.retail)
    
    ons_df = ons_df.assign(education_and_professional_activities = ons_df.education + 
                                                        ons_df.professional_scientific_and_technical_activities)
    ons_df = ons_df.assign(utilities = ons_df.electricity_gas_steam_and_air_conditioning_supply + 
                           ons_df.water_supply_sewerage_waste_and_remediation_activities)
    ons_df = ons_df.assign(personal_and_public_services = ons_df.real_estate_activities + 
                                                    ons_df['public_admin_and_defence;_compulsory_social_security'] +
                                                    ons_df.other_service_activities)
    sic_letters.loc['wholesale_retail_motor_trade_and_repair'] = 'G'
    sic_letters.loc['wholesale_and_retail'] = 'G46_47'
    sic_letters.loc['education_and_professional_activities'] = 'M_P'
    sic_letters.loc['utilities'] = 'D_E'
    sic_letters.loc['personal_and_public_services'] = 'L_O_S'
    sic_letters.loc['others'] = 'Z'
    sic_letters.loc['vacancies'] = 'vacancies'
    #
    ons_df.month = pd.to_datetime(ons_df.month)
    # only need vacancies within a certain period
    ons_df = ons_df[(ons_df.month>=pd.to_datetime(start_date)) & 
                    (ons_df.month<=pd.to_datetime(end_date))]
    ons_df = ons_df.set_index('month')
    return ons_df, sic_letters
'''
print('Uncomment if from flow_to_stock_funcs import get_stock_breakdown does not work')


In [ ]:
# All credit for this function goes to https://github.com/oliviaguest/gini/blob/master/gini.py
def gini(array):
    """Calculate the Gini coefficient of a numpy array."""
    # based on bottom eq:
    # http://www.statsdirect.com/help/generatedimages/equations/equation154.svg
    # from:
    # http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    # All values are treated equally, arrays must be 1d:
    array = array.flatten()
    if np.amin(array) < 0:
        # Values cannot be negative:
        array -= np.amin(array)
    # Values cannot be 0:
    array += 0.0000001
    # Values must be sorted:
    array = np.sort(array)
    # Index per array element:
    index = np.arange(1,array.shape[0]+1)
    # Number of array elements:
    n = array.shape[0]
    # Gini coefficient:
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array)))

# Load data

## Job adverts dataset

In [ ]:
print('Usually would load Textkernel dataset')
data_df = []


## Load data on TTWAs and other geographies

1. TTWA dictionary to go from code to name
2. TTWA statistics
3. TTWA shapefiles
4. Regions shapefiles
5. Country shapefiles


In [ ]:
ttwa_folder = '/path/to/ttwa/data/ONS/Travel_to_Work_Areas_2011_guidance_and_information_V4'
ttwa_file_1 = 'TTWA-info-2016.xls'
ttwa_file_2 = 'TTWA-summary-statistics-2011.xls'
ttwa_stats = pd.read_excel(f"{ttwa_folder}/{ttwa_file_1}")
print('TTWA stats from 2016')
print(ttwa_stats.columns)
tmp = pd.read_excel(f"{ttwa_folder}/{ttwa_file_2}")
print('--------')
print('TTWA stats from 2011')
print(tmp.columns)
ttwa_stats = ttwa_stats.merge(tmp, on =['ttwa11cd','ttwa11nm'], how= 'left')
ttwa_code2name = dict(zip(ttwa_stats.ttwa11cd,ttwa_stats.ttwa11nm))
ttwa_name2code = dict(zip(ttwa_stats.ttwa11nm,ttwa_stats.ttwa11cd))


In [ ]:
ttwa_name2code['London']


In [ ]:
all_ttwas_codes = data_df.ttwa11cd.value_counts().index.to_list


In [ ]:
london_code = 'E30000234'


In [ ]:
biggest_ttwas = ttwa_stats.ttwa11cd[ttwa_stats[' Population']>60000].to_list() + [ttwa_name2code['Belfast']]
london_code in biggest_ttwas


## Load official stastistics of employment levels (economically active residents)

In [ ]:

employment_levels = pd.read_csv(f"{local_data_path}/aux/economically_active_15_19_16plus_by_ttwa.csv", skiprows=6)
employment_levels = employment_levels.rename(columns={'mnemonic': 'ttwa11cd'}).set_index('ttwa11cd').iloc[:-3]
employment_levels = employment_levels[[t for t in employment_levels.columns if 'Jan' in t]]
employment_levels = employment_levels.rename(columns=
                                {t: f"employment_{t[-4:]}" for t in employment_levels.columns if 'Jan' in t})
employment_levels


In [ ]:
# Need to add statistics for Northern Ireland
# No data is available for TTWAs - so, get statistics ...
# ...at the country level and project on TTWAs based on the distribution by TTWA as obtained from the census
NI_col = 'Number of economically active residents (aged 16+)'
NI_ttwa = ttwa_stats[ttwa_stats['Region/Country']=='Northern Ireland'][[NI_col,'ttwa11cd']]
NI_active = pd.read_csv(f"{local_data_path}/aux/economically_active_16plus_11_15_19_NI.csv", skiprows=7)
NI_active = NI_active.iloc[1:]
NI_active['Date'] = NI_active['Date'].map(lambda x: x[-4:])
NI_active = NI_active.set_index('Date')
NI_ratio_2011 = {}
for year in ['2015','2016','2017','2018','2019']:
    NI_ratio_2011[year] = float(NI_active.loc[year,'Northern Ireland'])/float(
        NI_active.loc['2011','Northern Ireland'])
    NI_ttwa[f"employment_{year}"] = NI_ttwa[NI_col]*NI_ratio_2011[year]
NI_ttwa = NI_ttwa.set_index('ttwa11cd')
NI_ttwa = NI_ttwa[[t for t in NI_ttwa.columns if 'employment' in t]]
printdf(NI_ttwa.head())
employment_levels = pd.concat((employment_levels,NI_ttwa))
employment_levels.head()


In [ ]:
# Select TTWAs with population size larger than specific thresholds. How big are these subsets of TTWAs?
biggest_employment_ttwas40 = employment_levels[employment_levels.mean(axis=1)>40000].index.to_list()
print(len(biggest_employment_ttwas40))
print(len(biggest_ttwas))
biggest_employment_ttwas50 = employment_levels[employment_levels.mean(axis=1)>50000].index.to_list()
print(len(biggest_employment_ttwas50))
biggest_employment_ttwas250 = employment_levels[employment_levels.mean(axis=1)>250000].index.to_list()
print(london_code in biggest_employment_ttwas250, london_code in biggest_employment_ttwas40)
print(ttwa_name2code['Belfast'] in biggest_employment_ttwas250)
[ttwa_code2name[t] for t in biggest_employment_ttwas250]


## Load information on most representative cluster for each job advert

In [ ]:
# release memory
data_df += ['most_representative_cluster']


## Load SOC to skill cluster crosswalks

In [ ]:

with gzip.GzipFile(f"{DATA_PATH}/data/aux/final_all_crosswalks_soc_to_clusters_top_avg_{DATE_ID}.gz",'rb') as f:
    soc_by_clusters = pickle.load(f)

# make sure everything is a float and not an int
for k1 in soc_by_clusters.keys():
    for k2 in soc_by_clusters[k1].keys():
        soc_by_clusters[k1][k2] = soc_by_clusters[k1][k2]+0.0

# normalise all crosswalk so that it sums up to 1 for each occupation
timer.start_task('normalising crosswalks')
soc_by_clusters_norm = {}
for year in soc_by_clusters.keys():
    soc_by_clusters_norm[year] = {}
    for k in soc_by_clusters[year].keys():
        tmp = deepcopy(soc_by_clusters[year][k])
        tmp = tmp/tmp.sum()
        soc_by_clusters_norm[year][k] = tmp
        
timer.end_task()


## Load information on the skills taxonomy


In [ ]:
# Load the mapping from job adverts skills to ESCO-based skill clusters
RESULTS_PATH = '/path/to/skills/taxonomy'
validated_matches_file = f'tk_skills_to_skills_and_clusters_validated_final_{DATE_ID}.csv'
final_matches_file = f'tk_skills_to_clusters_1to1_final_{DATE_ID}.csv'

tk_esco_121 = pd.read_csv(f"{RESULTS_PATH}/{final_matches_file}", encoding = 'utf-8')
tk_esco_121 = tk_esco_121.set_index('skill_label')

# turn dataframe into dictionary
tk_esco_121_dict = dict(zip(tk_esco_121.index,tk_esco_121.cluster_level_3))
  


In [ ]:
# reload the ESCO-based skill clusters
# Load full clusters
esco_clusters = pd.read_csv(esco_clusters_file)
# make alt labels list
esco_clusters['alt_labels'] = esco_clusters.alt_labels.map(
        lambda x: x.split('\n') if isinstance(x,str) else [])

# adjustments to the labels
esco_clusters.loc[esco_clusters.level_2==20.0,'label_level_2'] = 'land transport (rail)'
esco_clusters.loc[esco_clusters.level_3==191.0,'label_level_3'
                 ] = 'leather production (manufacturing)'
esco_clusters.loc[esco_clusters.level_3==190.0,'label_level_3'] = 'footwear design'
esco_clusters.loc[esco_clusters.level_3==57.0,'label_level_3'] = 'marketing (branding)'

print('All ESCO skills: ', len(esco_clusters))
print('Valid ESCO skills: ',len(esco_clusters[esco_clusters.level_1<15]))

print(esco_clusters.columns)



In [ ]:
# load cluster labels
cluster_labels_1 = pd.read_csv(f"{esco_clusters_dir}/ESCO_Essential_clusters_Level_1.csv")
cluster_labels_2 = pd.read_csv(f"{esco_clusters_dir}/ESCO_Essential_clusters_Level_2.csv")
cluster_labels_3 = pd.read_csv(f"{esco_clusters_dir}/ESCO_Essential_clusters_Level_3.csv")

cluster_labels_3 = cluster_labels_3.set_index('level_3')
cluster_labels_2 = cluster_labels_2.set_index('level_2')

# correct for same labels!!
cluster_labels_3.loc[191,'label'] = 'leather production (manufacturing)'
cluster_labels_3.loc[190,'label'] = 'footwear design'
cluster_labels_3.loc[57,'label'] = 'marketing (branding)'
cluster_labels_2.loc[20,'label'] = 'land transport (rail)'



In [ ]:
# build the crosswalks between levels in the skills taxonomy
esco_first_to_second={}
esco_second_to_first={}
for name,g in esco_clusters.groupby('level_1').level_2:#.value_counts():
    level_2_all = sorted(g.value_counts().index.to_list())
    esco_first_to_second[name] = level_2_all
    for level_id in level_2_all:
        esco_second_to_first[level_id] = name
    
esco_second_to_third={}
esco_third_to_second = {}
esco_third_to_first = {}
for name,g in esco_clusters.groupby('level_2').level_3:#.value_counts():
    level_3_all = sorted(g.value_counts().index.to_list())
    esco_second_to_third[name] = level_3_all
    for level_id in level_3_all:
        esco_third_to_second[level_id] = name
        esco_third_to_first[level_id] = esco_second_to_first[name]

esco_first_to_third = {}
for name in esco_first_to_second.keys():
    level_2_all = esco_first_to_second[name]
    level_3_all = []
    for level_id in level_2_all:
        level_3_all.append(esco_second_to_third[level_id])
    esco_first_to_third[name] = sorted(flatten_lol(level_3_all))

print('done')

# same but with the levels
esco_first_to_second_label={}
esco_second_to_first_label={}
for name,g in esco_clusters.groupby('label_level_1').label_level_2:#.value_counts():
    level_2_all = sorted(g.value_counts().index.to_list())
    esco_first_to_second_label[name] = level_2_all
    for level_id in level_2_all:
        esco_second_to_first_label[level_id] = name
    
esco_second_to_third_label={}
esco_third_to_second_label = {}
esco_third_to_first_label = {}
for name,g in esco_clusters.groupby('label_level_2').label_level_3:#.value_counts():
    level_3_all = sorted(g.value_counts().index.to_list())
    esco_second_to_third_label[name] = level_3_all
    for level_id in level_3_all:
        esco_third_to_second_label[level_id] = name
        esco_third_to_first_label[level_id] = esco_second_to_first_label[name]

esco_first_to_third_label = {}
for name in esco_first_to_second_label.keys():
    level_2_all = esco_first_to_second_label[name]
    level_3_all = []
    for level_id in level_2_all:
        level_3_all.append(esco_second_to_third_label[level_id])
    esco_first_to_third_label[name] = sorted(flatten_lol(level_3_all))

print('done')


## Load ONS vacancy data

In [ ]:
# Load ONS stock of vacancies
raw_jvs_full, jvs_sic_letters = load_ons_vacancies()
# Change all the columns names
raw_jvs_full = raw_jvs_full.rename(columns = {t: jvs_sic_letters.loc[t] for t in jvs_sic_letters.index})

# Select desired industries and average across time
raw_jvs_avg = raw_jvs_full[['B', 'C', 'D_E', 'F', 'G', 'H', 'I',
        'J', 'K', 'L_O_S', 'M_P', 'N', 'O', 'Q', 'R']].mean()
raw_jvs_avg['A'] = np.nan
raw_jvs_avg['T'] = np.nan
raw_jvs_avg = raw_jvs_avg.T




# Compute stock breakdowns


Quick easy math about the stock


In [ ]:
# how many combinations could we expect in total?
N_soc = len(data_df.profession_soc_code_value.value_counts())
N_ttwa = len(data_df.ttwa11cd.value_counts())
N_sic = len(data_df.final_sic_letter.value_counts())
N_months = 12*3 + 10 + 10


In [ ]:
print((f"Total combinations: {N_soc}*{N_ttwa}*{N_sic}*{N_months} = "
       f"{N_soc*N_ttwa*N_sic*N_months}, for {len(data_df)} job adverts"))

print((f"Nb of combinations per month: {N_soc}*{N_ttwa}*{N_sic} = "
       f"{N_soc*N_ttwa*N_sic}, for {len(data_df)/N_months:.2f} job adverts "
      "on average per month"))



In [ ]:
# Resample SOC codes
data_df['profession_soc_code_1'] = resample_soc_to_n_digits_df(
    data_df.profession_soc_code_value, n=1)

data_df['profession_soc_code_2'] = resample_soc_to_n_digits_df(
    data_df.profession_soc_code_value, n=2)

data_df['profession_soc_code_3'] = resample_soc_to_n_digits_df(
    data_df.profession_soc_code_value, n=3)


## Compute pre-resampling stock by variuos breakdowns

In [ ]:
# Get all possible partial stock breakdowns
timer.start_task('Get all the RAW stocks')
raw_stocks_per_month = {}
possible_breakdowns = [['final_sic_letter'], ['profession_soc_code_value'], ['ttwa11cd']]

keys_breakdowns = ['sic_only','soc_only','ttwa_only']

COMPUTE_RAW_STOCKS = True
SAVE_RAW_STOCKS = False
if COMPUTE_RAW_STOCKS:
    for i,breakdown_of_interest in enumerate(possible_breakdowns):
        raw_stock_per_month_partial, _, _ , _ = get_stock_breakdown(
                data_df[['final_sic_letter', 'ttwa11cd','profession_soc_code_value',
                         'vacancy_weight_new','date', 'end_date']],
                agg_func = 'count', 
                agg_col = 'vacancy_weight_new',
                breakdown_col = breakdown_of_interest)
        raw_stocks_per_month[keys_breakdowns[i]] = raw_stock_per_month_partial
        timer.end_task()
    if SAVE_RAW_STOCKS:
        with open(f"{DATA_PATH}/data/aux/raw_stock_per_month_partial_all_{DATE_ID}.pickle",'wb') as f:
            pickle.dump(raw_stocks_per_month,f)
else:
    with open(f"{DATA_PATH}/data/aux/raw_stock_per_month_partial_all_{DATE_ID}.pickle",'rb') as f:
        raw_stocks_per_month = pickle.load(f)

for k in raw_stocks_per_month.keys():
    raw_stocks_per_month[k] = raw_stocks_per_month[k].iloc[2:]
    
        

## Compute post sampling stock by various breakdowns

In [ ]:
# Get all possible partial stock breakdowns
timer.start_task('Get all the stocks')
stocks_per_month = {}
possible_breakdowns = [['final_sic_letter'], ['profession_soc_code_value'], ['ttwa11cd'],
                       ['profession_soc_code_1'],['profession_soc_code_2'],['profession_soc_code_3'],
                      ['final_sic_letter','ttwa11cd'],
                      ['ttwa11cd','profession_soc_code_value'],
                       ['ttwa11cd','profession_soc_code_1'],
                      ['ttwa11cd','profession_soc_code_2'],
                      ['ttwa11cd','profession_soc_code_3']]
                        
keys_breakdowns = ['sic_only','soc4_only','ttwa_only','soc1_only','soc2_only','soc3_only',
                    'sic_ttwa','ttwa_soc4','ttwa_soc1','ttwa_soc2','ttwa_soc3']

COMPUTE_STOCKS = True
SAVE_STOCKS = False
if COMPUTE_STOCKS:
    for i,breakdown_of_interest in enumerate(possible_breakdowns):
        stock_per_month_partial, _, _ , _ = get_stock_breakdown(
                data_df[['final_sic_letter', 'ttwa11cd','region_label',
                         'profession_soc_code_value','profession_soc_code_1',
                         'profession_soc_code_2','profession_soc_code_3',
                         'vacancy_weight_new','date', 'end_date']],
                agg_func = 'sum', 
                agg_col = 'vacancy_weight_new', 
                breakdown_col = breakdown_of_interest)
        stocks_per_month[keys_breakdowns[i]] = stock_per_month_partial
        timer.end_task()
    if SAVE_STOCKS:
        with open(f"{DATA_PATH}/data/aux/stock_per_month_partial_all_{DATE_ID}.pickle",'wb') as f:
            pickle.dump(stocks_per_month,f)
else:
    with open(f"{DATA_PATH}/data/aux/stock_per_month_partial_all_{DATE_ID}.pickle",'rb') as f:
        stocks_per_month = pickle.load(f)


## remove first two months
for k in stocks_per_month.keys():
    stocks_per_month[k] = stocks_per_month[k].iloc[2:]


# Analysis of estimates of the stock of skill demand

1. Representativeness of job adverts by industry: a) what are the biggest differences before and after? b) Stock over time for Agriculture and Households
2. Stock of skill demand by occupation
3. Stock of skill demand by location. Then regional variations in skill demand by industry.
4. Stock of skill demand by location. Then regional variations in skill demand by industry.


## Stock of vacancies by industry: representativeness

In [ ]:
# compute the stock before and after adjustment by ONS vacancies
sic_codes_to_use = [col for col in stocks_per_month['sic_only'].columns if 'uncertain' not in col]

sic_after = stocks_per_month['sic_only'][
    sic_codes_to_use].mean()

sic_before = raw_stocks_per_month['sic_only'][sic_codes_to_use].mean()



In [ ]:
# Join everything in one dataframe, together with ONS vacancies
full_sic_df = pd.DataFrame(sic_before).merge(pd.DataFrame(sic_after), right_index = True, left_index= True).merge(
    pd.DataFrame(raw_jvs_avg.iloc[2:]), right_index= True, left_index= True)

full_sic_df = full_sic_df.rename(columns = {'0_x': 'Pre-resampling stock',
                                            '0_y': 'Post-resampling stock',
                                            0: 'ONS stock'})
full_sic_df = full_sic_df/full_sic_df.sum(axis=0)*100
full_sic_df = full_sic_df.sort_values(by= 'Post-resampling stock')

# Plot stock of vacancies before and after, together with ONS vacancies
with sns.plotting_context('talk'):
    f = full_sic_df[['Pre-resampling stock','ONS stock','Post-resampling stock']].rename(
        index = lambda x: sic_letter_to_text_abbr[x]).plot(kind='barh', figsize = (16,8),
        width = .8, color = [nesta_colours[t] for t in [1,5,2]])
    plt.tight_layout()
    plt.ylabel('Industry sector')
    plt.xlabel('Stock of vacancies (%)')
    if SAVEFIG:
        plt.savefig(f"{output_folder}/stock_distribution_by_sector_vs_ons.svg")
        plt.savefig(f"{output_folder}/stock_distribution_by_sector_vs_ons.jpg")
    


In [ ]:
# Plot percentages of stock by industry, in increasing order
full_sic_df.rename(sic_letter_to_text_abbr)['Post-resampling stock']



In [ ]:
# Join the stock of vacancies before and after adjustment, then print as a DataFrame
sic_both = pd.DataFrame(zip(sic_before.values,sic_after.values), columns =['before','after'],
                        index = sic_before.index.map(lambda x: sic_letter_to_text_abbr[x]))
sic_both = sic_both.sort_values('before', ascending=False)
sic_both['after'] =sic_both['after'].astype('int')
sic_both['before'] =sic_both['before'].astype('int')

sic_quotient = []
total_after= sic_both['after'].sum()
total_before = sic_both['before'].sum()
for sic in sic_both.index:
    sic_row= sic_both.loc[sic]
    #print(sic,sic_row)
    weight_after = sic_row['after']/(total_after)
    weight_before = sic_row['before']/(total_before)
    #print(sic,weight_before,weight_after)
    sic_quotient.append(weight_before/weight_after)
    
sic_both = sic_both.assign(sic_quotient = sic_quotient)
sic_both.after = np.around(sic_both.after/sic_both.after.sum()*100,2)
sic_both.sic_quotient = sic_both.sic_quotient.map(lambda x: np.around(x,2))
sic_both.index.name = 'Industry sector'
sic_both[['sic_quotient','after']].sort_values(by='sic_quotient', ascending=False).rename(
    columns = {'sic_quotient': 'Ratio (pre-/post-adjustment stock)',
              'after': 'Vacancies share (%)'}).iloc[:-1]



### Stock of vacancies by industry: Agriculture and Households


In [ ]:
# plot "missing sectors" over time
with sns.plotting_context('talk'):
    fig, ax = plt.subplots(figsize = (15,6))
    stocks_per_month['sic_only'].rename_axis('Date', axis = 0, inplace=True)
    stocks_per_month['sic_only'][['A']].rename(columns = {'A': sic_letter_to_text['A']}).plot(
        ax=ax, linewidth = 2, color = 'k')#nesta_colours[2])
    ax.set_xticks([543,546,549,552,555,558,561,564,567,570,
                   573,576,579,582,585,588,591,594,597])
    plt.ylabel('Stock of vacancies (Agriculture)')
    plt.ylim([700,2700])
    stocks_per_month['sic_only'][['T']].rename(columns = {'T': 'Acitivites of households as employers'}).plot(
        ax= ax, secondary_y = True, linewidth = 2, color = 'r')
    plt.ylabel('Stock of vacancies (Households)')
    
    plt.tight_layout()
    if SAVEFIG:
        plt.savefig(f"{output_folder}/stock_over_time_agriculture_households3.svg")
        plt.savefig(f"{output_folder}/stock_over_time_agriculture_households3.png")
    
    

In [ ]:
# Load employment by industry, for comparison
employment_industry = pd.read_csv(
    '/Users/stefgarasto/Local-Data/scripts/skill_demand_escoe/skill_demand/data/'+
    'aux/employment_by_industry_15_19_all_UK.csv',
     skiprows = 7)
employment_industry = employment_industry[[t for t in employment_industry.columns if not 'Conf' in t]]
employment_industry.set_index('Date')[['T13a:1 (A Agricuture & fishing (SIC 2007) : All people )',
                                         'T13a:25 (R-U Other services (SIC 2007) : All people )']]
employment_industry = employment_industry.rename(columns = {
    'T13a:25 (R-U Other services (SIC 2007) : All people )': 'R_S_T_U',
    'T13a:1 (A Agricuture & fishing (SIC 2007) : All people )': 'A',
    'T13a:4 (B,D,E Energy & water (SIC 2007) : All people )': 'B_D_E',
    'T13a:7 (C Manufacturing (SIC 2007) : All people )': 'C',
    'T13a:10 (F Construction (SIC 2007) : All people )': 'F',
    'T13a:13 (G,I Distribution, hotels & restaurants (SIC 2007) : All people )': 'G_I',
    'T13a:16 (H,J Transport & Communication (SIC 2007) : All people )': 'H_J',
    'T13a:19 (K-N Banking finance & insurance etc. (SIC 2007) : All people )': 'K_M_N',
    'T13a:22 (O-Q Public admin education & health (SIC 2007) : All people )': 'O_P_Q'
})

a = ['B_D_E','C','F','G_I','H_J','K_M_N','O_P_Q']
print('Employment by industry')
print(employment_industry[a].mean()/employment_industry[a].mean().sum()*100)

# Join up ONS vacancy data by industries based on our custom groupings
tmp = raw_jvs_full[['B','C','D_E','F','G','I','H','J','K','M','N','O','P','Q','R','S']].mean()
tmp['G_I'] = tmp['G'] + tmp['I']
tmp = tmp.drop(['G','I'])
tmp['H_J'] = tmp['H'] + tmp['J']
tmp = tmp.drop(['H','J'])
tmp['B_D_E'] = tmp['B'] + tmp['D_E']
tmp = tmp.drop(['B','D_E'])
tmp['K_M_N'] = tmp['K'] + tmp['M'] + tmp['N']
tmp = tmp.drop(['K','M','N'])
tmp['O_P_Q'] = tmp['O'] + tmp['P'] + tmp['Q']
tmp = tmp.drop(['O','P','Q'])
tmp = tmp.drop(['R','S'])

print('ONS vacancies by industries')
print(100*tmp/tmp.sum())


## Stock by occupation




### Composition of stock by occupations, averaged across time and comparison with employment

In [ ]:
def load_ons_employment(ons_filename):
    if ons_filename == 'employment_by_occupation1_15_19_in_UK.csv':
        skiprows= 10#43
        nrows = 10
        n=1
    elif ons_filename == 'employment_by_occupation2_15_19_in_UK.csv':
        skiprows = 10#59
        nrows = 26
        n=2
    elif ons_filename == 'employment_by_occupation3_15_19_in_UK.csv':
        skiprows = 10#124
        nrows = 91
        n=3
    elif ons_filename == 'employment_by_occupation4_15_19_in_UK.csv':
        skiprows = 9#124
        nrows = 370
        n=4
    employment_by_soc = pd.read_csv(f"{local_data_path}/aux/{ons_filename}",
                                 skiprows= skiprows, nrows = nrows)
    
    try:
        employment_by_soc = employment_by_soc.set_index('Occupation')
        employment_by_soc = employment_by_soc[[t for t in employment_by_soc.columns if not 'Conf' in t]]
        employment_by_soc = employment_by_soc.rename(columns = {t: f"employment {t[-4:]}" 
                                                                for t in employment_by_soc.columns})
        employment_by_soc = employment_by_soc.iloc[1:]
        #employment_by_soc = employment_by_soc/employment_by_soc.sum()*100
    except:
        print('not good')
        printdf(employment_by_soc.head())
    return employment_by_soc
    
employment_by_soc1 = load_ons_employment('employment_by_occupation1_15_19_in_UK.csv')
employment_by_soc2 = load_ons_employment('employment_by_occupation2_15_19_in_UK.csv')
employment_by_soc3 = load_ons_employment('employment_by_occupation3_15_19_in_UK.csv')
employment_by_soc4 = load_ons_employment('employment_by_occupation4_15_19_in_UK.csv')



In [ ]:
# composition by 1-digit soc codes
active_stock = deepcopy(stocks_per_month['soc1_only'].resample('Y').mean().T)
# normalise each year to get shares
print(100*active_stock.mean(axis=1).values/employment_by_soc1.mean(axis=1).values)
active_stock = active_stock/active_stock.sum()*100
active_stock = active_stock.rename(columns = 
                                   {pd.to_datetime(f"{t}-12-31"): t 
                                    for t in ['2015','2016','2017','2018','2019']})
active_stock['Occupation'] = active_stock.index.map(lambda x: socnames_dict[x])
active_stock['Avg share (adverts)'] = active_stock[
    [t for t in active_stock.columns if not 'Occupation' in t]].mean(axis=1)
tmp_ons = deepcopy(employment_by_soc1)
tmp_ons= np.around(tmp_ons/tmp_ons.sum()*100,2)
tmp_ons['Avg share (ONS)'] = tmp_ons.mean(axis=1).values
tmp_ons['soc_code'] = tmp_ons.index.map(lambda x: float(x[:1]))

active_stock = active_stock.merge(tmp_ons[['soc_code','Avg share (ONS)']], left_index= True, right_on = 'soc_code')

printdf(active_stock[['Occupation','Avg share (adverts)','Avg share (ONS)']].head(20))

# Print in a format that can easily copy-pasted to Excel to make tables
for i in ['Occupation','Avg share (adverts)','Avg share (ONS)']:
    for t in active_stock[i].values:
        if i=='Occupation':
            print(t)
        else:
            print(f"{t:.2f}")
    print('---')
for t in active_stock[i].index:
        print(t[:1])



In [ ]:
# composition by n-digit soc codes
n= 4
active_stock = deepcopy(stocks_per_month[f'soc{n}_only'].resample('Y').mean().T)
# normalise each year to get shares
if n==1:
    employment_of_interest = employment_by_soc1
elif n==2:
    employment_of_interest = employment_by_soc2
elif n ==3:
    employment_of_interest = employment_by_soc3
elif n==4:
    employment_of_interest = employment_by_soc4
else:
    stop

#print(100*active_stock.mean(axis=1).values/employment_of_interest.mean(axis=1).values)
active_stock = active_stock/active_stock.sum()*100
active_stock = active_stock.rename(columns = 
                                   {pd.to_datetime(f"{t}-12-31"): t for t in ['2015','2016','2017','2018','2019']})
active_stock['Occupation'] = active_stock.index.map(lambda x: socnames_dict[x])
active_stock['Avg share (adverts)'] = active_stock[
    [t for t in active_stock.columns if not 'Occupation' in t]].mean(axis=1)
tmp_ons = deepcopy(employment_of_interest)
tmp_ons= np.around(tmp_ons/tmp_ons.sum()*100,2)
tmp_ons['Avg share (ONS)'] = tmp_ons.mean(axis=1).values
tmp_ons['soc_code'] = tmp_ons.index.map(lambda x: float(x[:n]))

active_stock = active_stock.merge(tmp_ons[['soc_code','Avg share (ONS)']], left_index= True, right_on = 'soc_code') 
printdf(active_stock[['Occupation','Avg share (adverts)','Avg share (ONS)']].head(20))

# Print in a format that can easily copy-pasted to Excel to make tables
for i in ['Occupation','Avg share (adverts)','Avg share (ONS)']:
    for t in active_stock[i].values:
        if i=='Occupation':
            print(t)
        else:
            print(f"{t:.2f}")
    print('---')
for t in active_stock[i].index:
        print(t[:n])





### SOC over time

In [ ]:
# Which occupations have changed the most over time?
tmp = stocks_per_month['soc1_only'].resample('Y').mean().T
tmp = tmp/tmp.sum()*100
tmp = tmp[tmp.mean(axis=1)>.1]
tmp = tmp.assign(ratio_2019_2015 = 100*(tmp['2019-12-31']-tmp['2015-12-31'])/tmp['2015-12-31']).fillna(0)
tmp.index= tmp.index.map(lambda x: socnames_dict[x])

tmp = tmp.rename(columns = {pd.to_datetime(f'{t}-12-31'): f"{t} (%)" for t in ['2019','2018','2017','2016','2015']})
tmp = tmp.rename(columns = {'ratio_2019_2015': 'relative change 2015 to 2019 (%)'})
tmp.name = 'Occupation (1-digit SOC)'

printdf(tmp.applymap(lambda x: np.around(x,2)))


In [ ]:
# Breakdown of more granular occupations (share of vacancies)
stock_by_soc3 = stocks_per_month['soc3_only'].resample('Y').mean().T
# turn into share
stock_by_soc3 = stock_by_soc3/stock_by_soc3.sum()*100
# Only keep occupations with, on average, at least .1% share
stock_by_soc3 = stock_by_soc3[stock_by_soc3.mean(axis=1)>.1]
stock_by_soc3 = stock_by_soc3.assign(ratio_2019_2015 = 100*(stock_by_soc3['2019-12-31']-stock_by_soc3['2015-12-31']
                                       )/stock_by_soc3['2015-12-31']).fillna(0)

stock_by_soc3['profession_soc_code_1'] = stock_by_soc3.index.map(lambda x: resample_soc_to_n_digits(x,n=2))

# group by 1-digit SOC
soc1_groups = stock_by_soc3.groupby('profession_soc_code_1')

    

In [ ]:
# Plot minor groups grouped by Major groups - changes over time
for g in range(1,10):
    with sns.plotting_context('talk'):
        group1_2015 = soc1_groups.get_group(g).sort_values(
            pd.to_datetime('2015-12-31'), ascending=False)[pd.to_datetime('2015-12-31')]
        group1_2019 = soc1_groups.get_group(g).sort_values(
            pd.to_datetime('2015-12-31'), ascending=False)[pd.to_datetime('2019-12-31')]
        group1_max = group1_2015.sum()
        group1_2019_norm = group1_2019
        group1_2015_norm = group1_2015
        pd.DataFrame(zip(group1_2019_norm, group1_2015_norm), columns = ['2019','2015'],
                    index = group1_2015_norm.index.map(lambda x: socnames_dict[x])).T.plot(
            kind='barh', stacked=True, figsize = (15,5), color = aug_nesta_colours)#, ax=f.gca())
        plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
        plt.title(socnames_dict[g])
        plt.xlabel('Vacancies share (%)')
        plt.ylabel('Year')
        plt.tight_layout()
        if SAVEFIG:
            plt.savefig(f"{output_folder}/4d_breakdown_share_change_group{g}.svg")
            plt.savefig(f"{output_folder}/4d_breakdown_share_change_group{g}.png")

        

In [ ]:
print(soc1_groups.get_group(6).ratio_2019_2015.mean())
soc1_groups.get_group(6).sort_index()



## Stock of vacancies across TTWA

### Adjust by economically active residents and plot

In [ ]:
# Plot stock distribution by TTWA averaged across time
plt.close('all')
year = '2018'
row = {'2016': 1, '2017': 2, '2018': 3, '2019': 4}
# normalise by employment for each year then take the average across years
data_prep = stocks_per_month['ttwa_only'].resample('Y').mean().T

# join with relevant employment estimates
data_prep = data_prep.merge(employment_levels, left_on='ttwa11cd', right_index=True)

for year in ['2015','2016','2017','2018','2019']:
    year_index = pd.to_datetime(f"31-12-{year}")
    data_prep[year_index] = data_prep[year_index]/data_prep[f"employment_{year}"]*100
    data_prep = data_prep.rename(columns = {year_index: f'stock_{year}'})
data_prep['Stock'] = data_prep[[t for t in data_prep.columns if 'stock' in t]].mean(axis=1)
data_prep['employment_avg'] = data_prep[[t for t in data_prep.columns if 'employment' in t]].mean(axis=1)

data_prep = data_prep.reset_index()
data_prep = data_prep.fillna(0)

#Don't show TTWAs with less than 40,000 in employment
data_prep = data_prep[data_prep.ttwa11cd.map(lambda x:
                            x in biggest_employment_ttwas40)] 

# Plot map
w=8
fig,ax = plt.subplots(figsize = (w,14))
shp_folder = '/path/to/maps/files'
gb_filename = os.path.join(shp_folder,
                'Countries2016/Countries_December_2016_Ultra_Generalised_Clipped_Boundaries_in_Great_Britain.shp')
ni_filename = os.path.join(shp_folder,
                'OSNI_Open_Data_Largescale_Boundaries__Country_2016.shp')
shp_filename = os.path.join(shp_folder,
    'TTWA2011_full_clipped/Travel_to_Work_Areas_December_2011_Full_Clipped_Boundaries_in_United_Kingdom.shp')
# Plot
data_to_plot= pd.DataFrame(data_prep[['Stock','ttwa11cd']])
col_to_plot = 'Stock'
with sns.plotting_context('talk'):
    _, _, xs, ys = draw_map(data_to_plot, col_to_plot, 'GnBu', 
                        data_to_plot[col_to_plot].min(), 
                        data_to_plot[col_to_plot].max(), 
                        gb_filename, ni_filename, #gb_filename
                        shp_filename, roi_col = 'ttwa11cd',
                        shp_col = 'ttwa11cd',
                        fig = fig, ax= ax)

tmp = ax.set_title('Stock of vacancies per 100 economically active residents', fontsize = 18)

if SAVEFIG:
    plt.savefig(f"{output_folder}/avg_stock_by_100_employment_by_ttwa.svg", bbox_inches = 'tight')
    plt.savefig(f"{output_folder}/avg_stock_by_100_employment_by_ttwa.png", bbox_inches = 'tight')



In [ ]:
# Top and bottom TTWAs for vacancies per 100 people in employment
plt.close('all')
data_prep['ttwa11nm'] = data_prep['ttwa11cd'].map(lambda x: ttwa_code2name[x])
print('Bottom 5')
print(data_prep.sort_values('Stock', ascending = True).head(10).ttwa11nm.tolist())
printdf(data_prep.sort_values('Stock', ascending = True).head())
print('Top 5')
print(data_prep.sort_values('Stock', ascending = False).head(10).ttwa11nm.tolist())
printdf(data_prep.sort_values('Stock', ascending = False).head())
print(np.percentile(data_prep.Stock,1),np.percentile(data_prep.Stock,5),
     np.percentile(data_prep.Stock,95),np.percentile(data_prep.Stock,99))
print(data_prep.Stock.min(),data_prep.Stock.max())
print(np.percentile(data_prep.Stock,99)/np.percentile(data_prep.Stock,1))
print(data_prep.Stock.max()/data_prep.Stock.min())



### Stock by SIC x TTWA: which SIC codes are more evenly distributed across TTWA?

In [ ]:
# data preparation
data_to_use = deepcopy(stocks_per_month['sic_ttwa'])
data_prep = data_to_use.mean()
data_prep = data_prep
data_prep = data_prep.reset_index(level=1).rename(columns = {0:'stock'})
data_prep = data_prep.reset_index(drop=False)
data_prep = data_prep[data_prep.final_sic_letter!='uncertain']


In [ ]:
# rearrange stock by sic and ttwa from long to wide form
tmp = []
for sic in data_prep.final_sic_letter.value_counts().index:
    tmp.append(data_prep[data_prep.final_sic_letter==sic].stock.values)
all_ttwas = data_prep[data_prep.final_sic_letter=='A'].ttwa11cd

sic_by_ttwa = pd.DataFrame(tmp, columns = all_ttwas, 
             index= data_prep.final_sic_letter.value_counts().index)

# drop the part of stock that is not actually matched to a TTWA
sic_by_ttwa = sic_by_ttwa.drop(columns = ['not_found'])
sic_by_ttwa.head()



In [ ]:

# compute proportion of skill demand by sector across all UK
sic_value_counts = data_df.final_sic_letter.value_counts()
full_sector_in_uk = sic_by_ttwa.sum(axis=1)/sic_by_ttwa.sum().sum()
quotient_sic_ttwa = pd.DataFrame(columns = sic_by_ttwa.columns,
                                index = sic_by_ttwa.index)

# get the sector location quotients for all TTWAs. Note, this is computed using the WHOLE of the UK
for chosen_ttwa in sic_by_ttwa.columns:
    quotient_sic_ttwa[chosen_ttwa] = (sic_by_ttwa[chosen_ttwa]/sic_by_ttwa[chosen_ttwa].sum())/full_sector_in_uk

# only keep largest TTWAs (economically active residents >40K)
quotient_sic_ttwa = quotient_sic_ttwa[biggest_employment_ttwas40]

# now compute the Gini coefficient for each sector. This is computed ONLY on SELECTED TTWAs
gini_by_sector = []
for chosen_sic in sic_by_ttwa.index:
    gini_by_sector.append(gini(quotient_sic_ttwa.loc[chosen_sic].to_numpy()))

quotient_sic_ttwa= quotient_sic_ttwa.assign(gini_index = gini_by_sector)

quotient_sic_ttwa = quotient_sic_ttwa.assign(counts = sic_value_counts)

quotient_sic_ttwa.index = quotient_sic_ttwa.index.map(lambda x: sic_letter_to_text_abbr[x])

# show the London location quotients, from high to low
printdf(pd.DataFrame(quotient_sic_ttwa[london_code]).rename(
    columns = {'E30000234': 'London local quotient'}).sort_values(by = 'London local quotient', ascending= False))

# Add Cambridge location quotients
ttwaname = 'Cambridge'
printdf(pd.DataFrame(quotient_sic_ttwa[ttwa_name2code[ttwaname]]).rename(
    columns = {ttwa_name2code[ttwaname]: f'{ttwaname} local quotient'}).sort_values(
    by = f'{ttwaname} local quotient', ascending= False))


# Show Gini coefficients, high to low + ratio
rho_gini_counts= np.corrcoef(
    quotient_sic_ttwa[['gini_index','counts']].astype('float32').to_numpy().T)[0,1]
print((f"Correlation between gini index and sic value counts: "
       f"{rho_gini_counts:.2f}"))
print((f"Highest Gini index is {quotient_sic_ttwa.gini_index.sort_values().iloc[-2]:.2f}"))
print((f"Lowest Gini index is {quotient_sic_ttwa.gini_index.min():.2f}"))
print((f"Ratio between highest and lowest Gini index is "
      f"{quotient_sic_ttwa.gini_index.sort_values().iloc[-2]/quotient_sic_ttwa.gini_index.min():.2f}"))

pd.DataFrame(quotient_sic_ttwa['gini_index'].sort_values(
          ascending = False).rename('Gini index')).iloc[1:]



In [ ]:
# Plot location quotients for Information and communication and Administrative and support service activities
sic_letter2use = 'Information and communication'
sic_letter2use2 = 'Administrative and support service activities'

# plot best and worst sector next to each other in order
quotient_sic_ttwa_red = quotient_sic_ttwa.drop(['gini_index','counts'],axis= 1).T[[sic_letter2use,sic_letter2use2]]

# only keep the ones with > 250k economically active residents
quotient_sic_ttwa_red = quotient_sic_ttwa_red[
    quotient_sic_ttwa_red.index.map(lambda x: x in biggest_employment_ttwas250)]
with sns.plotting_context('talk'):
    quotient_sic_ttwa_red.sort_values(by = sic_letter2use).rename(lambda x: ttwa_code2name[x]).plot(
        kind = 'barh', figsize = (15,11), color = [nesta_colours[t] for t in [1,2]])
    plt.legend(loc='lower right', title= 'Industry')
    plt.plot([quotient_sic_ttwa_red['Information and communication'].min(),
              quotient_sic_ttwa_red['Information and communication'].min()],
            [-1,32], '--', color = nesta_colours[1])
    plt.plot([quotient_sic_ttwa_red['Information and communication'].max(),
              quotient_sic_ttwa_red['Information and communication'].max()],
            [-1,32], '--', color = nesta_colours[1])
    plt.ylabel('TTWA')
    plt.xlabel('Location quotient')
    plt.tight_layout()

print(quotient_sic_ttwa_red[sic_letter2use].max()/quotient_sic_ttwa_red[sic_letter2use].min())
print(quotient_sic_ttwa_red[sic_letter2use2].max()/quotient_sic_ttwa_red[sic_letter2use2].min())

if SAVEFIG:
    plt.savefig(f"{output_folder}/ttwa_quotients_IT_Admin.svg")
    plt.savefig(f"{output_folder}/ttwa_quotients_IT_Admin.png")


In [ ]:
# Print Top TTWAs for selected industries
quotient_sic_ttwa_red = quotient_sic_ttwa.drop(['gini_index','counts'],axis= 1).T
# Top TTWAs in Finance:
print('Top TTWAs in Finance')
print(quotient_sic_ttwa_red['Financial and insurance activities'].sort_values(ascending=False).head(10).rename(
    lambda x: ttwa_code2name[x]).index.tolist())
print('Top TTWAs in Agriculture')
print(quotient_sic_ttwa_red['Agriculture, forestry and fishing'].sort_values(ascending=False).head(10).rename(
    lambda x: ttwa_code2name[x]).index.tolist())
print('Top TTWAs in Utilities')
print(quotient_sic_ttwa_red['Utilities (energy, water and waste)'].sort_values(ascending=False).head(10).rename(
    lambda x: ttwa_code2name[x]).index.tolist())



## Stock of vacancies by TTWA and skill clusters


### Compute stock by skill cluster over the years using the crosswalk


In [ ]:
# Crosswalk from stock by occupation only (soc_by_clusters has been loaded earlier)
month_by_clusters = {}
for k_esco in ['level_1','level_2','level_3']:
    k = f'4d_by_{k_esco}'
    active_crosswalk = {}
    for year in soc_by_clusters.keys():
        active_crosswalk[year] = deepcopy(soc_by_clusters_norm[year][k].fillna(0))

    # crosswalk from stock by soc to stock by cluster
    month_by_cluster = pd.DataFrame(index= stocks_per_month['soc4_only'].index,
                                   columns = active_crosswalk[year].index)
    for month in stocks_per_month['soc4_only'].index:
        row = deepcopy(stocks_per_month['soc4_only'].loc[month,:]).to_numpy()
        row = row[:,np.newaxis]
        year_to_use = str(month.year)
        row = np.dot(active_crosswalk[year_to_use].fillna(0).to_numpy(),row)
        month_by_cluster.loc[month,:] = row[:,0]
    
    # check the total stock is roughly the same
    print(month_by_cluster.sum().sum()//1000, stocks_per_month['soc4_only'].sum().sum()//1000)
    
    month_by_clusters[k_esco] = month_by_cluster.astype('float')

SAVE_CLUSTER_STOCK = False
if SAVE_CLUSTER_STOCK:
    with gzip.GzipFile(f"{DATA_PATH}/data/aux/stock_by_month_by_cluster_all_{DATE_ID}.gz", 'wb') as f:
        pickle.dump(month_by_clusters,f)
        
    

In [ ]:
# stock for cluster level 1
stock_composition_1 = pd.DataFrame(month_by_clusters['level_1'].resample('Y').mean())
stock_composition_1.index = stock_composition_1.index.map(lambda x: str(x)[:4])
stock_composition_1 = stock_composition_1.T

level_1_composition = stock_composition_1/stock_composition_1.sum(axis=0)*100
level_1_composition = level_1_composition.drop(15) #iloc[:201]
level_1_composition.index = level_1_composition.index.map(lambda x: 
                                                    cluster_labels_1.loc[int(x)].label)
level_1_composition.index= level_1_composition.index.map(lambda x: x.capitalize())

level_1_composition = level_1_composition.sort_values('2015', ascending = False)
print(100*(level_1_composition['2019'] - level_1_composition['2015']) / level_1_composition['2015'])

# Define the colours for each cluster at level 1
colors_for_cluster_level_1 = {}
for ix,t in enumerate(level_1_composition.index):
    print(t)
    colors_for_cluster_level_1[t]= ix
for t in level_1_composition.mean(axis=1)/level_1_composition.mean(axis=1).sum():
    print(f"{100*t:.2f}")

# Plot the distribution over years
with sns.plotting_context('talk'):
    f = plt.figure()
    level_1_composition[['2019','2018','2017','2016','2015']].T.plot(
        kind='barh', stacked=True, figsize = (14,5.8), ax=f.gca(), color = aug_nesta_colours)
    plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
    plt.xlabel('Vacancies share (%)')
    plt.ylabel('Year')
    plt.tight_layout()
    if SAVEFIG:
        plt.savefig(f"{output_folder}/stock_by_cluster_level1_2015_2019.png")
        plt.savefig(f"{output_folder}/stock_by_cluster_level1_2015_2019.svg")



In [ ]:
BASE_YEAR = '2015'


In [ ]:

# stock for cluster level 2
stock_composition_2 = pd.DataFrame(month_by_clusters['level_2'].resample('Y').mean())
stock_composition_2.index = stock_composition_2.index.map(lambda x: str(x)[:4])
stock_composition_2 = stock_composition_2.T

level_2_composition = stock_composition_2/stock_composition_2.sum(axis=0)*100
level_2_composition = level_2_composition.drop(76) #iloc[:201]
level_2_composition.index = level_2_composition.index.map(lambda x: 
                                                    cluster_labels_2.loc[int(x)].label)

level_2_composition.index = level_2_composition.index.map(lambda x: x.capitalize())

level_2_composition['mean_value'] = level_2_composition.mean(axis=1)
level_2_composition = level_2_composition.sort_values('mean_value', ascending = False)

larger_rows_2 = (level_2_composition.mean(axis=1)>0.9) #| (level_2_composition.index == 'emergency healthcare')

level_2_composition = level_2_composition.drop(columns = 'mean_value')


In [ ]:
# plot the composition by cluster level 2 (coloured by cluster level 1)
f = plt.figure(figsize = (11,12))
tmp = deepcopy(level_2_composition.loc[larger_rows_2,['2015','2016','2017','2018','2019']].reset_index())
tmp = tmp.melt(id_vars = 'row_0', value_vars = ['2015','2016','2017','2018','2019'], var_name= 'Year')

with sns.plotting_context('talk'):
    sns.barplot(x = 'value', y = 'row_0', hue= 'Year',
            data= tmp, ax = f.gca(), palette = sns.color_palette([nesta_colours[t] for t in [6,4,10,8,7]]))
    
    plt.ylabel('Second level skills clusters')
    plt.xlabel('Vacancies share (%)')
    plt.tight_layout()
    for t in f.gca().get_yticklabels():
        level_up = esco_second_to_first_label[t.get_text().lower()]
        color_id = colors_for_cluster_level_1[level_up.capitalize()]
        t.set_color(nesta_colours[color_id])
        #f.gca().set_yticklabels(t)#print(level_up,color_id)
    if SAVEFIG:
        plt.savefig(f"{output_folder}/stock_by_cluster_level2_2015_2019.png")
        plt.savefig(f"{output_folder}/stock_by_cluster_level2_2015_2019.svg")
    


In [ ]:
# top decreasing and increasing clusters among the ones with a least 1% share
# increase and decrease is taken as the average change across consecutive years or ...
# ...as the change between the average of consecutive years
active_composition = deepcopy(level_2_composition[larger_rows_2])
tmp = active_composition

for base_year in ['2015','2016','2017','2018']:
    next_year = f"{int(base_year)+1}"
    tmp[f'relative_change_{base_year}'] = (active_composition[next_year] - active_composition[base_year]
                                 )/active_composition[base_year]
tmp['average_2015_2016']= (tmp['2015'] + tmp['2016'])/2
tmp['average_2018_2019']= (tmp['2018'] + tmp['2019'])/2
tmp['relative_change_1516_1819'] = (tmp['average_2018_2019'] - tmp['average_2015_2016'])/tmp['average_2015_2016']
tmp['relative_change_avg'] = tmp[[f'relative_change_{t}' for t in ['2015','2016','2017','2018']]].mean(axis=1)
tmp['relative_change_std'] = tmp[[f'relative_change_{t}' for t in ['2015','2016','2017','2018']]].std(axis=1)

top5_relative_changes = tmp.sort_values('relative_change_avg').tail(5).index.to_list()
bottom5_relative_changes = tmp.sort_values('relative_change_avg').head(5).index.to_list()
top5_erratic_changes = tmp.sort_values('relative_change_std').tail(5).index.to_list()

100*tmp.sort_values(by = 'relative_change_avg', ascending= False)



In [ ]:
# Composition of stock by cluster level 3
stock_composition_3 = pd.DataFrame(month_by_clusters['level_3'].resample('Y').mean())
stock_composition_3.index = stock_composition_3.index.map(lambda x: str(x)[:4])
stock_composition_3 = stock_composition_3.T


level_3_composition = stock_composition_3/stock_composition_3.sum(axis=0)
level_3_composition = level_3_composition.drop(201) #iloc[:201]
level_3_composition.index = level_3_composition.index.map(lambda x: 
                                                    cluster_labels_3.loc[int(x)].label)



In [ ]:
# clusters at level 3 with largest share of demand
larger_rows_3 = (level_3_composition.mean(axis=1)>0.3/100)


In [ ]:
# show changes for some example clusters
for t in ['digital marketing', 'marketing', 'marketing (branding)', 'product management']:
    decrease_2015 = (level_3_composition.loc[t]['2019'] - level_3_composition.loc[t]['2015']
                    )/level_3_composition.loc[t]['2015']
    decrease_2016 = (level_3_composition.loc[t]['2019'] - level_3_composition.loc[t]['2016']
                    )/level_3_composition.loc[t]['2016']
    print(t, decrease_2015)


In [ ]:
skillset_to_check = list(set(top5_relative_changes + bottom5_relative_changes + top5_erratic_changes))

plt.close('all')

# show changes for more example clusters
for skillset in cluster_labels_2.label.tolist():
    if skillset == 'nan':
        continue
    sub_skillset = []
    for t in esco_second_to_third_label[skillset]:
        if t in level_3_composition.index:
            sub_skillset.append(t)

    select_level_3 = 100*level_3_composition.loc[sub_skillset].T
    if skillset== 'marketing & product management':
        select_level_3.columns = ['digital marketing','marketing','marketing (branding)',
                                                    'product management']
    max_tmp = select_level_3.loc['2015'].sum()
    if max_tmp<0.1:
        continue
    #select_level_3 = select_level_3/max_tmp
    with sns.plotting_context('talk'):
        select_level_3.sort_index(ascending=False).plot(kind='barh', stacked=True, figsize = (13,5), 
                                             color = aug_nesta_colours)
        plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
        plt.title(f"{skillset.capitalize()}")
        plt.xlabel('Vacancies share (%)')
        plt.ylabel('Year')
        plt.tight_layout()
        if SAVEFIG:
            plt.savefig(f"{output_folder}/stock_by_cluster_{skillset}_2015_2019.png")
            plt.savefig(f"{output_folder}/stock_by_cluster_{skillset}_2015_2019.svg")
        

### Compute and analyse diversity of skill clusters across TTWAs

In [ ]:
# crosswalk from (stock by ttwa x soc) to (stock by ttwa x skill cluster)
''' For each month take the distribution by ttwa and soc, multiply the distribution of stock
by soc within each ttwa by the crosswalk soc x clusters, then sum across the soc.
The result is the stock by ttwa and cluster for each month'''

timer.start_task()

month_by_ttwa_by_clusters = {}
month_by_ttwa_by_clusters_df = {}
which_soc = 3
for k_esco in ['level_3','level_2','level_1']:
    print(k_esco)
    k = f'{which_soc}d_by_{k_esco}'
    active_crosswalk = {}
    for year in soc_by_clusters.keys():
        active_crosswalk[year] = deepcopy(soc_by_clusters_norm[year][k].fillna(0)).T
        
    month_by_ttwa_by_cluster = {}
    month_by_ttwa_by_cluster_df = {}
    for i,month in enumerate(stocks_per_month[f'ttwa_soc{which_soc}'].index):
        #month_key
        if i%10==9:
            print(month)
        year_to_use = str(month.year)
        month_by_ttwa_by_cluster[month] = {}
        rows = deepcopy(stocks_per_month[f'ttwa_soc{which_soc}'].loc[month].reset_index(level=0))
        for ttwa in list(set(rows.index)):
            if ttwa == 'not_found':
                continue
            row2 = rows.loc[ttwa].reset_index(drop=True)
            #row2['profession_soc_code_3'] = row2['profession_soc_code_3'].astype('category')
            row2 = row2.set_index(f'profession_soc_code_{which_soc}')#value')
            row2_merged = deepcopy(row2.merge(active_crosswalk[year_to_use],
                                                                         left_index= True, 
                                                                         right_index= True).fillna(0))
            
            # perform dot product
            row_by_cluster = row2_merged.loc[:,row2_merged.columns[1:202]].mul(
                row2_merged[month],axis=0).sum()
            month_by_ttwa_by_cluster[month][ttwa] = row_by_cluster
        
        # combine into  a dataframe
        month_by_ttwa_by_cluster_df[month] = pd.DataFrame.from_dict(
            month_by_ttwa_by_cluster[month]).stack()
        
    # save for future
    month_by_ttwa_by_clusters[k_esco] = month_by_ttwa_by_cluster
    month_by_ttwa_by_clusters_df[k_esco] = month_by_ttwa_by_cluster_df

timer.end_task()
SAVE_CLUSTER_TTWA_STOCK = False
if SAVE_CLUSTER_TTWA_STOCK:
    # save it
    with gzip.GzipFile(f"{DATA_PATH}/data/aux/stock_by_month_by_ttwa_by_cluster_all_{DATE_ID}.gz", 'wb') as f:
        pickle.dump((month_by_ttwa_by_clusters,month_by_ttwa_by_clusters_df),f)
        


### Regional variation within skill clusters


In [ ]:
def get_local_quotients_and_gini(data_to_use, col_to_use, biggest_employment_ttwas, data_df, 
                                 is_value_counts= False, indices_dict = None):
    """ Compute location quotients for TTWAs, then compute the Gini coefficient of those location quotients
    for each value of the variable of interest"""
    # average across time
    data_prep = data_to_use.mean()
    # split the two dimensions across indices and columns
    data_prep = data_prep.reset_index(level=1).rename(columns = {0:'stock'})
    data_prep = data_prep.reset_index(drop=False)
    
    if not 'ttwa11cd' in data_prep.columns:
        data_prep = data_prep.rename(columns = {'level_1': 'ttwa11cd'})
    tmp = []
    i=0
    # var is the relevant dimension (SOC, skill cluster, industry, etc.)
    for var in data_prep[col_to_use].value_counts().index:
        tmp.append(data_prep[data_prep[col_to_use]==var].stock.values)
        if i==0:
            all_ttwas = data_prep[data_prep[col_to_use]==var].ttwa11cd
        i+=1
    #all_ttwas = data_prep[data_prep[col_to_use]==11.0].ttwa11cd
    var_by_ttwa = pd.DataFrame(tmp, columns = all_ttwas, 
                 index= data_prep[col_to_use].value_counts().index)

    # compute proportion of skill demand by "variable" across all UK
    full_dist_in_uk = var_by_ttwa.sum(axis=1)/var_by_ttwa.sum().sum()
    quotient_var_ttwa = pd.DataFrame(columns = var_by_ttwa.columns,
                                    index = var_by_ttwa.index)

    # get the location quotients for all TTWAs
    for chosen_ttwa in var_by_ttwa.columns:
        quotient_var_ttwa[chosen_ttwa] = (var_by_ttwa[chosen_ttwa]/var_by_ttwa[chosen_ttwa].sum()
                                         )/full_dist_in_uk

    # only keep largest TTWAs (population more than 60k)
    quotient_var_ttwa = quotient_var_ttwa[biggest_employment_ttwas]

    # now compute the Gini coefficient for each "variable"
    gini_by_var = []
    for chosen_var in var_by_ttwa.index:
        gini_by_var.append(gini(quotient_var_ttwa.loc[chosen_var].to_numpy()))

    quotient_var_ttwa= quotient_var_ttwa.assign(gini_index = gini_by_var)
    
    if is_value_counts:
        quotient_var_ttwa = quotient_var_ttwa.assign(counts = data_df)
    else:
        quotient_var_ttwa = quotient_var_ttwa.assign(counts = data_df[col_to_use].value_counts())

    if indices_dict:
        quotient_var_ttwa.index = quotient_var_ttwa.index.map(lambda x: indices_dict[x])

    print('Done')
    
    return var_by_ttwa, quotient_var_ttwa



In [ ]:
# Here, get the local quotients for each skillset in each TTWA - and then get the Gini index of the collection
# of local quotients across TTWA for each skillset (at variuos levels of granularity)
larger_rows_2_id = []
for name in larger_rows_2.index:
    if larger_rows_2.loc[name]:
        ix = cluster_labels_2[cluster_labels_2.label==name.lower()].index[0]
        larger_rows_2_id.append(ix)
larger_rows_2_id = sorted(larger_rows_2_id)

larger_rows_3_id = []
for name in larger_rows_3.index:
    if larger_rows_3.loc[name]:
        ix = cluster_labels_3[cluster_labels_3.label==name.lower()].index[0]
        larger_rows_3_id.append(ix)
larger_rows_3_id = sorted(larger_rows_3_id)

# cluster level 3
skill3_by_ttwa, quotient_skill3_ttwa = get_local_quotients_and_gini(
    pd.DataFrame.from_dict(month_by_ttwa_by_clusters_df['level_3']).loc[larger_rows_3_id].T, 
    'index', biggest_employment_ttwas40, stock_composition_3.mean(axis = 1), 
                is_value_counts = True, indices_dict = None)

# cluster level 2
skill2_by_ttwa, quotient_skill2_ttwa = get_local_quotients_and_gini(
    pd.DataFrame.from_dict(month_by_ttwa_by_clusters_df['level_2']).loc[larger_rows_2_id].T, 
    'index', biggest_employment_ttwas40, stock_composition_2.mean(axis = 1), 
                is_value_counts = True, indices_dict = None)

# cluster level 1
skill1_by_ttwa, quotient_skill1_ttwa = get_local_quotients_and_gini(
    pd.DataFrame.from_dict(month_by_ttwa_by_clusters_df['level_1']).T, 
    'index', biggest_employment_ttwas40, stock_composition_1.mean(axis = 1), 
                is_value_counts = True, indices_dict = None)



In [ ]:
# remove small clusters
largest_clusters_2 = skill2_by_ttwa.T.sum()>0
print('Number of clusters level 2 included: ',largest_clusters_2.sum())

largest_clusters_3 = skill3_by_ttwa.T.sum()>0
print('Number of clusters level 3 included: ',largest_clusters_3.sum())



In [ ]:
# distribution of Gini indices
with sns.plotting_context('talk'):
    # clusters level 3
    plt.figure()
    sns.distplot(quotient_skill3_ttwa.loc[largest_clusters_3,'gini_index'].dropna(), color = nesta_colours[2])
    print('Clusters level 3')
    plt.ylabel('Skill clusters counts')
    plt.xlabel('Gini index')
    plt.tight_layout()
    if SAVEFIG:
        plt.savefig(f"{output_folder}/gini_indices_for_skill_clusters_level3_over_ttwa.svg")
        plt.savefig(f"{output_folder}/gini_indices_for_skill_clusters_level3_over_ttwa.png")
    
    # clusters level 2
    plt.figure()
    sns.distplot(quotient_skill2_ttwa.loc[largest_clusters_2,'gini_index'].dropna(), color = nesta_colours[2])
    print('Clusters level 2')
    plt.ylabel('Skill clusters counts')
    plt.xlabel('Gini index')
    plt.tight_layout()
    if SAVEFIG:
        plt.savefig(f"{output_folder}/gini_indices_for_skill_clusters_level2_over_ttwa.svg")
        plt.savefig(f"{output_folder}/gini_indices_for_skill_clusters_level2_over_ttwa.png")
    
    

In [ ]:
# highest Gin indices at level 3: these are the skill clusters with higher variation across the UK
print('Clusters level 3. Highest Gini indices')
clusters_high_gini = quotient_skill3_ttwa.loc[largest_clusters_3,['gini_index','counts']].sort_values('gini_index', 
    ascending=False).head(10).rename(index = lambda x: cluster_labels_3.loc[x].label)

print(clusters_high_gini.index.tolist())
printdf(clusters_high_gini)


# lowest Gini indices at level 3: these are the skill clusters with lower variation across the UK
print('Lowest Gini indices')
clusters_low_gini =quotient_skill3_ttwa.loc[largest_clusters_3, ['gini_index','counts']].sort_values('gini_index', 
    ascending=True).head(10).rename(index = lambda x: cluster_labels_3.loc[x].label)
print(clusters_low_gini.index.tolist())

printdf(clusters_low_gini)

# print ratio of max gini over min gini
print(quotient_skill3_ttwa.loc[largest_clusters_3].gini_index.max(),
     quotient_skill3_ttwa.loc[largest_clusters_3].gini_index.min())
max_over_min = quotient_skill3_ttwa.loc[largest_clusters_3].gini_index.max(
    )/quotient_skill3_ttwa.loc[largest_clusters_3].gini_index.min()
print(f'Max/min is {max_over_min}')



In [ ]:
# highest Gini indices at level 2: these are the skill clusters with higher variation across the UK
quotient_skill2_ttwa[['gini_index','counts']
                    ][quotient_skill2_ttwa.counts>0].sort_values('gini_index', ascending=False).rename(
    index = lambda x: cluster_labels_2.loc[x].label)



In [ ]:
# all Gini indices at level 1, in decreasing order
quotient_skill1_ttwa[['gini_index','counts']
                    ][quotient_skill1_ttwa['counts']>10000].sort_values('gini_index', ascending=False).rename(
    index = lambda x: cluster_labels_1.loc[x].label)



In [ ]:
# how do location quotients for skill clusters correlate with those for industries?
industry_by_cluster = pd.merge(quotient_sic_ttwa.T, quotient_skill2_ttwa.T, 
                               left_index=True, right_index= True, how='inner')

variables_to_keep = industry_by_cluster.loc['counts']>5000

industry_by_cluster = industry_by_cluster[variables_to_keep[variables_to_keep].index]

industry_by_cluster = industry_by_cluster.drop(['gini_index','counts']).rename(columns = 
                    {x: cluster_labels_2.loc[x].label for x in cluster_labels_2.index}).corr()

industry_by_cluster_red = industry_by_cluster.iloc[:21,21:]

with sns.plotting_context('talk'):
    f = plt.figure(figsize = (18,11))
    sns.heatmap(industry_by_cluster_red.applymap(lambda x: np.around(x,2)), ax = f.gca(), 
                cmap = sns.color_palette("RdYlBu", 10), vmin=-1, vmax = 1)#, annot = True, fmt = '.2f')
    plt.tight_layout()
    if SAVEFIG:
        plt.savefig(f"{output_folder}/industry_cluster_level_2_locquotient_correlation.png")
        plt.savefig(f"{output_folder}/industry_cluster_level_2_locquotient_correlation.svg")
        
    

In [ ]:
industry_by_cluster = pd.merge(quotient_sic_ttwa.T, quotient_skill1_ttwa.T, 
                               left_index=True, right_index= True, how='inner')

variables_to_keep = industry_by_cluster.loc['counts']>5000
industry_by_cluster = industry_by_cluster[variables_to_keep[variables_to_keep].index]

industry_by_cluster = industry_by_cluster.drop(['gini_index','counts']).rename(columns = 
                    {x: cluster_labels_1.loc[x].label for x in cluster_labels_1.index}).corr()

industry_by_cluster_red = industry_by_cluster.iloc[:15,15:]

with sns.plotting_context('talk'):
    f = plt.figure(figsize = (14,10))
    sns.heatmap(industry_by_cluster_red.applymap(lambda x: np.around(x,2)), ax = f.gca(), 
                cmap = sns.color_palette("RdYlBu", 10), vmin=-1, vmax = 1)#, annot = True, fmt = '.2f')
    plt.tight_layout()
    if SAVEFIG:
        plt.savefig(f"{output_folder}/industry_cluster_level_1_locquotient_correlation.png")
        plt.savefig(f"{output_folder}/industry_cluster_level_1_locquotient_correlation.svg")


In [ ]:
# Print top TTWAs for some skills clusters level 2
for cluster_id in quotient_skill2_ttwa.sort_values(by='counts', ascending=False).head(40).index:
    print(f"TTWA with highest local quotient for second level cluster '{cluster_labels_2.loc[cluster_id].label}'")
    tmp = quotient_skill2_ttwa.loc[cluster_id].sort_values(ascending=False).drop(['gini_index','counts']).rename(
        lambda x: ttwa_code2name[x])
    print(tmp.head(5).index.tolist())
    print()

    

In [ ]:
# Print top TTWAs for some clusters level 1
for cluster_id in quotient_skill1_ttwa.sort_values(by='counts', ascending=False).head(10).index:
    print(f"TTWA with highest local quotient for second level cluster '{cluster_labels_1.loc[cluster_id].label}'")
    tmp = quotient_skill1_ttwa.loc[cluster_id].sort_values(ascending=False).drop(['gini_index','counts']).rename(
        lambda x: ttwa_code2name[x])
    print(tmp.head(5).index.tolist())
    print()
    

In [ ]:
test_variable = quotient_skill2_ttwa.rename(lambda x: cluster_labels_2.loc[x].label)
for select_ttwa in ['London','Aberdeen','Belfast','Edinburgh']:
    print(select_ttwa)
    print(test_variable[ttwa_name2code[select_ttwa]].sort_values(ascending= True).head(10))
    print(test_variable[ttwa_name2code[select_ttwa]].sort_values(ascending= False).head(10))
    print('-'*70)



# Analysis of skills within skill clusters

Skill cluster to check: 
[programming, Assembly and maintenance, import & export management, digital marketing, logistics, 'food preparation', 'aquaculture work', 'user interface', 'metal processing', 'curriculum management', 'policy', 'public relations & campaigning', 'import & export management', 'human resources', 'forensics']

In [ ]:
clusters_for_demand = ['programming', 'assembly & maintenance', 'import & export management', 
                       'digital marketing', 'marketing', 'logistics', 'general core skills','subject teaching']
clusters_for_demand += ['warehousing', 'subject teaching', 'machine operation', 'programming', 'food preparation', 
                      'web development', 'nursing', 'warehouse management & distribution', 'law', 'ict security',
                      'aquaculture work']
clusters_for_demand += ['shop management', 'logistics', 'merchandising', 'sales', 'secretarial assistance', 
                     'call centre services', 'business management', 'insurance', 'surveying',
                     'import & export management', 
                    'human resources', 'forensics']
for skill_clus in clusters_for_demand:
    # print most common 15 skills
    top_skills = tk_esco_121[tk_esco_121.cluster_label_level_3==skill_clus].sort_values(
        by='tk_counts', ascending=False).head(15).index.to_list()
    print(skill_clus)
    print(top_skills)
    print('-'*70)
    print()


# Save the supplementary material

## Composition by industry, location, occupation

In [ ]:
# print composition by industry, location, occupation
# Note that job adverts for which data is missing are not included when computing shares.
# Mining is also removed because there are too few job adverts matched to this industry

tables_folder = '/path/to/tables'
for k in ['sic_only', 'ttwa_only', 'soc1_only', 'soc2_only', 'soc3_only', 'soc4_only']:
    active_stock = stocks_per_month[k].resample('Y').mean().T
    if k=='ttwa_only':
        active_stock = active_stock.rename(columns = {pd.to_datetime(f"{t}-12-31"): f"{t}" 
                                    for t in ['2015','2016','2017','2018','2019']})
        # Divide by 100 economically active residents
        active_stock= active_stock.drop(['not_found'])
        active_stock = active_stock.merge(employment_levels, left_on='ttwa11cd', right_index=True)
        for year in ['2015','2016','2017','2018','2019']:
            year_index = pd.to_datetime(f"31-12-{year}")
            active_stock[year] = active_stock[year]/active_stock[f"employment_{year}"]*100
            active_stock = active_stock.rename(columns = {year: f'{year} (normalised stock)'})
        active_stock = active_stock[[f"{year} (normalised stock)" for year in 
                                     ['2015','2016','2017','2018','2019']]]
        # only select the biggest TTWAs
        active_stock = active_stock[active_stock.index.map(lambda x:
                            x in biggest_employment_ttwas40)] 
        
        active_stock = active_stock.rename(ttwa_code2name)
        savename = 'annual_normalised_stock_of_demand_by_ttwa'
        df_name = 'TTWA name'
        active_stock = active_stock.rename_axis(df_name, axis = 'index')
    else:
        active_stock = active_stock.rename(columns = {pd.to_datetime(f"{t}-12-31"): f"{t} (%)" 
                                    for t in ['2015','2016','2017','2018','2019']})
        if k == 'sic_only':
            active_stock= active_stock.drop(['uncertain','B'])
            active_stock = active_stock.rename(sic_letter_to_text)
            sheet_name = 'vacancies by industry'
            savename = 'annual_composition_of_demand_by_industry'
            df_name = 'SIC 2007'
        elif 'soc' in k:
            group_type = {'soc1_only': 'major_groups', 'soc2_only': 'sub-major_groups', 
                          'soc3_only': 'minor_groups', 'soc4_only': 'unit_groups'}[k]
            savename = f'annual_composition_of_demand_by_occupation_{group_type}'
            sheet_name = f"vacancies by {group_type.replace('_',' ')}"
            df_name = 'Occupation'
        active_stock =active_stock/active_stock.sum()*100
        active_stock = active_stock.rename_axis(df_name, axis = 'index')
    
    # round to 2 digits
    active_stock = active_stock.applymap(lambda x: np.around(x,2))
    # save
    #active_stock.to_csv(f"{tables_folder}/{savename}.csv")
    
    if k=='ttwa_only':
        active_stock.to_csv(f"{tables_folder}/{savename}.csv")
    elif 'soc' in k:
        tmp = group_type.replace('_',' ')
        active_stock[f"SOC 2010 ({tmp})"] = active_stock.index.map(lambda x: int(x))
        active_stock = active_stock.rename(socnames_dict).reset_index()
        active_stock = active_stock[[f"SOC 2010 ({tmp})",'Occupation','2015 (%)','2016 (%)',
                                    '2017 (%)','2018 (%)','2019 (%)']]
        active_stock.to_csv(f"{tables_folder}/{savename}.csv", index= False)
    else:
        if k == 'sic_only':
            continue
        active_stock.to_csv(f"{tables_folder}/{savename}.csv")
    



## Composition by skill clusters


In [ ]:
for k in month_by_clusters.keys():
    
    active_stock = pd.DataFrame(month_by_clusters[k].resample('Y').mean().T)
    active_stock = active_stock.rename(columns = {pd.to_datetime(f"{t}-12-31"): f"{t} (%)" 
                                for t in ['2015','2016','2017','2018','2019']})
    cluster_to_drop = {'level_1':15, 'level_2': 76, 'level_3': 201}[k]
    active_stock = active_stock.drop(cluster_to_drop)
    cluster_labels_active = {'level_1': cluster_labels_1, 'level_2': cluster_labels_2,
                            'level_3': cluster_labels_3}[k]
    active_level = k[-1]
    active_stock = active_stock.rename(lambda x: cluster_labels_active.loc[int(x)].label.capitalize())
    sheet_name = 'vacancies by skill category'
    savename = f'annual_composition_of_demand_by_skill_category_level_{active_level}'
    df_name = f'Skill category (level {active_level})'
    active_stock =active_stock/active_stock.sum()*100
    active_stock = active_stock.rename_axis(df_name, axis = 'index')
    
    active_stock = active_stock.sort_values(by = '2015 (%)', ascending = False)
    #save column names
    
    # round to 2 digits
    active_stock = active_stock.applymap(lambda x: np.around(x,2))
    # save
    active_stock.to_csv(f"{tables_folder}/{savename}.csv")
    
    #active_stock.to_csv(f"{tables_folder}/{savename}.csv")
    printdf(active_stock.head())


# [Internal use only] Get SIC by SOC breakdown from job adverts matched via TK


In [ ]:
data_to_add = pd.read_csv(f"{DATA_PATH}/data/interim/interim_job_id_and_vacancy_weights.gz", 
    compression = 'gzip', encoding = 'utf-8', usecols = ['posting_id','organization_industry_value'])



In [ ]:

def get_map_industry_label_values():
    """ Get a map defining which TK ID value for sector corresponds to which label"""
    tmp_data = pd.read_csv(os.path.join(f"{DATA_PATH}/data/processed", 
                                        tk_params.file_name_template.format(0)), 
                           compression='gzip',
                encoding = 'utf-8',usecols = ['organization_industry_label',
                                              'organization_industry_value'])
    map_label2value = {}
    map_value2label = {}
    for name,g in tmp_data.groupby('organization_industry_value'):
        map_value2label[name] = g.organization_industry_label.value_counts().index.values[0]
        map_label2value[map_value2label[name]] = name
    return map_label2value, map_value2label

# create the maps
map_label2value, map_value2label = get_map_industry_label_values()



In [ ]:
# Define which TK categories are best matched to SIC codes (g = good, a =ambiguous)
assessment_map_tk2sic = {5: 'g', 
                      19: 'g',
                      11 : 'g',
                      8: 'g',
                      13: 'g',
                      6: 'g',
                      9: 'g',
                      10: 'g', 
                    12: 'g',
                    1: 'g',
                    2:'g',
                     16:'a',
                    7:'g',
                    14:'g',
                    21:'a',
                    18:'g',
                    17:'a',
                    4:'a',
                    15:'a',
                    20:'u',
                    0: 'u',
                    3: 's'}

In [ ]:
data_to_add['industry_type_label'] = data_to_add.organization_industry_value.map(
    lambda x: assessment_map_tk2sic[x])
print('done')

In [ ]:
data_to_add = data_to_add.merge(data_df[['posting_id','profession_soc_code_value',
                                         'final_sic_letter']], 
                                on ='posting_id', how='left')
print('done')


In [ ]:
sic_soc_ga_breakdown = data_to_add[data_to_add.industry_type_label.map(lambda x: x in ['g','a'])]
sic_soc_ga_breakdown = pd.crosstab(sic_soc_ga_breakdown['profession_soc_code_value'],
                                  sic_soc_ga_breakdown['final_sic_letter'])


In [ ]:
sic_soc_ga_breakdown.to_csv(
    '/Users/stefgarasto/Local-Data/textkernel/results/sic_matches/sic_soc_breakdown_from_select_adverts.csv')


In [ ]:
# normalise to 1000 top
sic_soc_ga_breakdown = 1000*sic_soc_ga_breakdown/sic_soc_ga_breakdown.max()
# save again
sic_soc_ga_breakdown.to_csv(
    '/Users/stefgarasto/Local-Data/textkernel/results/sic_matches/sic_soc_breakdown_from_select_adverts_to_share.csv')
